#BandMath & CloudMask

Queries can retrieve one or more variables (e.g. bands) and it is possible to perform opperations on them. 


In [1]:
%matplotlib inline
from datetime import  date
from pprint import pprint
from gdf import dt2secs, GDF 
from analytics import Analytics
from execution_engine import ExecutionEngine
from analytics_utils import plot

In [2]:
'''
Start and end year. With LS5 and data available on 
the virtual machine it works with years 2010 and 2011
'''
start_year=2010 #int(input("Ingrese el año inicial/ start year"))
end_year=2010 #int(input("Ingrese el año final/ end year"))

In [3]:
a = Analytics()
e = ExecutionEngine()

start_date = dt2secs(date(year=start_year,month=1,day=1))  #Start_year-01-01
end_date = dt2secs(date(year=end_year, month=12, day=31)) #End_year-12-31

dimensions = {'X': {'range': (147.0, 147.125)},
              'Y': {'range': (-36.0, -36 +0.125)},
              'T': {'range': (start_date, end_date)}
              }

'''
It should work with LS7ETM y LS7ETMPQ, 
however we doesn't know the valid ranges 
for this dataset at this moment.
'''
arrays = a.createArray('LS5TM', ['B40','B30'], dimensions, 'bands_data') #B30: Band 3, B40: Band 4
pq_data = a.createArray('LS5TMPQ', ['PQ'], dimensions, 'pq_data') #PQ: Pixel Quality, variable that will be used in masking. 

DEBUG:analytics:Initialise Analytics Module.


Initialise Analytics Module.


INFO:gdf:Forcing refresh of all cached data


Forcing refresh of all cached data


INFO:gdf:Connected to databases ['landsat', 'modis']


...

variable = B30


DEBUG:analytics:storage_type = LS5TMPQ


storage_type = LS5TMPQ


DEBUG:analytics:variable = PQ


variable = PQ


In [ ]:
##Operations between bands
ndvi = a.applyBandMath(arrays, '((array1 - array2) / (array1 + array2))', 'ndvi') #Normalized Difference Vegetation Index

##Cloud masking using quality of pixel data
mask = a.applyCloudMask(ndvi, pq_data, 'mask') #Cloud mask

#Reduction (mean)
median_t = a.applyGenericReduction(mask, ['T'], 'median(array1)', 'medianT') 


DEBUG:analytics:function before = ((array1 - array2) / (array1 + array2))


function before = ((array1 - array2) / (array1 + array2))


DEBUG:analytics:function after = ((B40 - B30) / (B40 + B30))


function after = ((B40 - B30) / (B40 + B30))


DEBUG:analytics:function before = median(array1)


function before = median(array1)


DEBUG:analytics:function after = median(mask)


function after = median(mask)
{'mask': {'array_input': ['ndvi'],
          'array_mask': 'pq_data',
          'array_output': {'data_type': 'int16',
                           'dimensions': {'T': {'range': (1262304000,
                                                          1293753600)},
                                          'X': {'range': (147.0, 147.125)},
                                          'Y': {'range': (-36.0, -35.875)}},
                           'dimensions_order': ['T', 'Y', 'X'],
                           'no_data_value': -999.0,
                           'shape': (17, 500.0, 500.0),
                           'storage_type': 'LS5TM',
                           'variable': 'mask'},
          'function': 'apply_cloud_mask',
          'orig_function': 'apply_cloud_mask'}}
{'mask': {'array_input': ['ndvi'],
          'array_mask': 'pq_data',
          'array_output': {'data_type': 'int16',
                           'dimensions': {'T': {'range': (1262304000,
     

In [ ]:
result = e.executePlan(a.plan)
plot(e.cache['medianT'])
